In [1]:
import sys
import pandas as pd
import util_mwb
print(sys.version)
print(pd.__version__) # Must be 0.24.1 for IntegerArray

3.7.5 (default, Oct 25 2019, 15:51:11) 
[GCC 7.3.0]
0.25.2


In [2]:
# Display all columns - slow; takes up a lot of memory so commented out
#pd.options.display.max_columns = None
#pd.options.display.float_format = '{:,.2f}'.format
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
%%time
csl_df = util_mwb.read_csl_csv()
csl_df

CPU times: user 2min 9s, sys: 1.14 s, total: 2min 10s
Wall time: 2min 10s


,Sitenum,MomID,Marital,Education,Insurance,Momage,Parity,Gravidity,Hxlivebirth,Hxmacrosomia,...,Fulldilattime,Placenttime,Inducttime,Ripentime,ROMtime,Epiduraltime,Intraantibtime,USdays,LMPdays,onepregflag
0,41,41-00001,1,3,1,34,4,5,4,1,...,15,4,361,NaN,300,343,346,103,271,2
1,41,41-00001,1,3,1,36,5,6,5,2,...,18,3,369,NaN,251,314,NaN,128,278,2
2,41,41-00002,1,3,1,37,1,3,1,1,...,NaN,1,NaN,NaN,1,4,NaN,128,268,1
3,41,41-00003,1,3,1,36,1,4,1,1,...,69,4,382,NaN,369,364,NaN,133,273,1
4,41,41-00004,1,2,1,34,2,6,2,1,...,12,5,339,NaN,105,116,455,204,274,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228433,52,52-07247,8,1,8,19,0,1,0,9,...,86,5,1236,471,464,403,NaN,NaN,260,1
228434,52,52-07248,8,8,8,34,1,3,1,9,...,NaN,NaN,168,1580,NaN,182,NaN,NaN,175,1
228435,52,52-07249,8,8,8,28,3,5,2,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
228436,52,52-07250,8,8,8,16,0,1,0,9,...,NaN,5,NaN,NaN,NaN,248,NaN,NaN,NaN,1


In [4]:
csl_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228438 entries, 0 to 228437
Columns: 779 entries, Sitenum to onepregflag
dtypes: Int16(38), Int32(2), Int8(663), float64(59), object(17)
memory usage: 450.1+ MB


In [5]:
csl_df.isnull().sum()

Sitenum                0
MomID                  0
Marital                1
Education              1
Insurance              1
                   ...  
Epiduraltime      124528
Intraantibtime    181958
USdays            163192
LMPdays           121799
onepregflag            0
Length: 779, dtype: int64

In [6]:
csl_df.describe(include='all')

,Sitenum,MomID,Marital,Education,Insurance,Momage,Parity,Gravidity,Hxlivebirth,Hxmacrosomia,...,Fulldilattime,Placenttime,Inducttime,Ripentime,ROMtime,Epiduraltime,Intraantibtime,USdays,LMPdays,onepregflag
count,228438,228438,228437.000000,228437.000000,228437.000000,228115.000000,228438.000000,226720.000000,160448.000000,228438.000000,...,145591.000000,192847.000000,65530.000000,10917.000000,76061.000000,103910.000000,46480.000000,65246.000000,106639.000000,228438.000000
unique,12,208695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,41,45-01637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,50320,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.970184,6.432855,2.081563,27.627946,1.139894,2.667665,1.084626,4.170786,...,72.767960,8.262643,567.211323,958.151598,437.929885,308.309922,381.678442,145.074165,269.954848,1.168632
std,NaN,NaN,1.457665,2.865649,2.083136,6.196163,1.367558,1.826788,1.280143,3.634488,...,211.966159,71.324187,535.209984,613.710233,659.183549,255.902863,606.139353,59.235567,18.760556,0.374428
min,NaN,NaN,1.000000,1.000000,1.000000,11.000000,0.000000,1.000000,0.000000,1.000000,...,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,NaN,NaN,1.000000,3.000000,1.000000,23.000000,0.000000,1.000000,0.000000,1.000000,...,13.000000,2.000000,233.000000,559.000000,106.000000,134.000000,105.000000,123.000000,266.000000,1.000000
50%,NaN,NaN,1.000000,8.000000,1.000000,27.000000,1.000000,2.000000,1.000000,1.000000,...,32.000000,4.000000,414.000000,846.000000,281.000000,252.000000,228.000000,143.000000,274.000000,1.000000
75%,NaN,NaN,3.000000,8.000000,2.000000,32.000000,2.000000,3.000000,2.000000,8.000000,...,79.000000,6.000000,715.000000,1213.000000,553.000000,419.000000,459.000000,193.000000,280.000000,1.000000


In [7]:
# Clear some memory
#import gc
#del csl_df
#gc.collect()